In [2]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

In [117]:
def getAllDoxyDonkeyPosts(url,links):
    request = Request(url)
    response = urlopen(request)
    soup = BeautifulSoup(response, "lxml")
    for a in soup.findAll('a'):
        try:
            url = a['href']
            title = a['title']
            if title == "Older Posts":
                links.append(url)
                getAllDoxyDonkeyPosts(url,links)
        except:
            title = ""
    return

blogUrl = "http://doxydonkey.blogspot.com"
links = []
getAllDoxyDonkeyPosts(blogUrl,links)

In [70]:
def getDoxyDonkeyText(testUrl):
    request = Request(testUrl)
    response = urlopen(request)
    soup = BeautifulSoup(response, "lxml")
    mydivs = soup.findAll("div", {"class":'post-body'})
    
    posts =[]
    for div in mydivs:
        posts+=map(lambda p:p.text.encode('ascii', errors='replace').replace(b"?",b" "), div.findAll("li"))
    return posts

In [71]:
doxyDonkeyPosts = []
for link in links:
    doxyDonkeyPosts+=getDoxyDonkeyText(link)

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [74]:
vectorizer = TfidfVectorizer(max_df=0.5,min_df=2,stop_words='english')

In [75]:
X = vectorizer.fit_transform(doxyDonkeyPosts)
X

<2804x13220 sparse matrix of type '<class 'numpy.float64'>'
	with 280835 stored elements in Compressed Sparse Row format>

In [76]:
X[0]

<1x13220 sparse matrix of type '<class 'numpy.float64'>'
	with 108 stored elements in Compressed Sparse Row format>

In [79]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 100, n_init = 1, verbose = True)

In [111]:
km.fit(X)

Initialization complete
Iteration  0, inertia 5188.507
Iteration  1, inertia 2686.429
Iteration  2, inertia 2681.372
Iteration  3, inertia 2678.465
Iteration  4, inertia 2675.570
Iteration  5, inertia 2674.003
Iteration  6, inertia 2673.083
Iteration  7, inertia 2672.594
Iteration  8, inertia 2672.189
Iteration  9, inertia 2671.787
Iteration 10, inertia 2671.368
Iteration 11, inertia 2671.100
Iteration 12, inertia 2670.796
Iteration 13, inertia 2670.472
Iteration 14, inertia 2670.234
Iteration 15, inertia 2669.951
Iteration 16, inertia 2669.529
Iteration 17, inertia 2668.835
Iteration 18, inertia 2667.720
Iteration 19, inertia 2667.289
Iteration 20, inertia 2667.074
Iteration 21, inertia 2666.804
Iteration 22, inertia 2666.524
Iteration 23, inertia 2666.318
Iteration 24, inertia 2666.181
Iteration 25, inertia 2666.071
Iteration 26, inertia 2666.001
Iteration 27, inertia 2665.932
Iteration 28, inertia 2665.896
Iteration 29, inertia 2665.858
Iteration 30, inertia 2665.836
Iteration 31, i

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=3, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=True)

In [112]:
import numpy as np
np.unique(km.labels_, return_counts=True)

(array([0, 1, 2], dtype=int32), array([1987,  390,  427]))

In [113]:
text={}
for i,cluster in enumerate(km.labels_):
    oneDocument = doxyDonkeyPosts[i].decode("utf-8")
    if cluster not in text.keys():
        text[cluster] = oneDocument
    else:
        text[cluster] += oneDocument

In [97]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest
import nltk 

In [98]:
_stopwords = set(stopwords.words('english') + list(punctuation)+["million","billion","year","millions","billions","y/y","'s","''","``"])

In [114]:
keywords = {}
counts={}
for cluster in range(3):
    word_sent = word_tokenize(text[cluster].lower())
    word_sent=[word for word in word_sent if word not in _stopwords]
    freq = FreqDist(word_sent)
    keywords[cluster] = nlargest(100, freq, key=freq.get)
    counts[cluster]=freq

In [115]:
unique_keys={}
for cluster in range(3):   
    other_clusters=list(set(range(3))-set([cluster]))
    keys_other_clusters=set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
    unique=set(keywords[cluster])-keys_other_clusters
    unique_keys[cluster]=nlargest(10, unique, key=counts[cluster].get)

In [101]:
unique_keys

{0: ['uber',
  'india',
  'round',
  'chinese',
  'funding',
  'capital',
  'raised',
  'investment',
  'e-commerce',
  'valuation'],
 1: ['ads',
  'video',
  'use',
  'products',
  'product',
  'search',
  'apps',
  'pay',
  'ad',
  'way'],
 2: ['quarter',
  'share',
  'profit',
  'rose',
  'earnings',
  'analysts',
  'cents',
  'per',
  'fell',
  'net']}

In [102]:
article = "Pinterest has raised $150 million in a new round of funding. The new deal values Pinterest at $12.3 billion, according to a company spokesperson, up from a valuation of $11 billion when it last raised funding two years ago.The round came from Pinterest’s existing investors, but the company is not saying which ones. Bloomberg first reported the news. Pinterest says it plans to use the money to help build its new visual search technology, which lets users employ images instead of keywords to find things on the service. The money will also be used to help grow its user base outside the United States. Around 40 percent of Pinterest’s users are in the U.S. Pinterest has raised well over $1 billion since it was founded seven years ago. The company’s business is finally starting to pick up; it expects to make more than $500 million in revenue this year, a 66 percent jump over last year, and more than Snapchat and Twitter made in the years before their respective IPOs."

In [110]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=20)
classifier.fit(X,km.labels_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=20, p=2,
           weights='uniform')

In [105]:
test=vectorizer.transform([article.encode('ascii',errors='ignore')])

In [106]:
test

<1x13220 sparse matrix of type '<class 'numpy.float64'>'
	with 68 stored elements in Compressed Sparse Row format>

In [116]:
classifier.predict(test)

array([1], dtype=int32)